In [1]:
from cmethods import CMethods as cm

In [2]:
import xarray as xr

In [10]:
obs = xr.open_dataset("examples/input_data/observations.nc")["tas"]
simh = xr.open_dataset("examples/input_data/control.nc")["tas"]
simp = xr.open_dataset("examples/input_data/scenario.nc")["tas"]

In [11]:
cm.adjust(method="quantile_delta_mapping",obs=obs,simh=simh,simp=simp, kind="+", n_quantiles=100)

<xarray.DataArray 'tas' (time: 10950, lat: 4, lon: 2)>
array([[[-23.10136938, -22.10136938],
        [-24.44140058, -23.44140058],
        [-26.01662437, -25.01662437],
        [-26.28269982, -25.28269982]],

       [[-23.41589465, -22.41589465],
        [-24.56309161, -23.56309161],
        [-24.04938832, -23.04938832],
        [-26.01993313, -25.01993313]],

       [[-23.18047936, -22.18047936],
        [-24.61759495, -23.61759495],
        [-24.38520952, -23.38520952],
        [-25.59637237, -24.59637237]],

       ...,

       [[-25.56795121, -24.56795121],
        [-26.43865472, -25.43865472],
        [-28.31595975, -27.31595975],
        [-29.04694175, -28.04694175]],

       [[-26.37948592, -25.37948592],
        [-27.46839831, -26.46839831],
        [-28.41172919, -27.41172919],
        [-29.84433331, -28.84433331]],

       [[-25.65145194, -24.65145194],
        [-28.06217378, -27.06217378],
        [-28.43720573, -27.43720573],
        [-29.98579602, -28.98579602]]])
Coordinates:
  * lat      (lat) int64 23 24 25 26
  * lon      (lon) int64 0 1
  * time     (time) object 2001-01-01 00:00:00 ... 2030-12-31 00:00:00

In [12]:
a=cm.adjust(method="delta_method",obs=obs,simh=simh,simp=simp,group="time.month",kind="+")
b=cm.adjust(method="delta_method",obs=obs,simh=simh,simp=simp,kind="+")


time.month


In [16]:
list(a)==list(b)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
def apply_ufunc(method, obs, simh, simp, **kwargs):
    result = xr.apply_ufunc(
        cm.delta_method,
        obs,
        simh,
        # Need to spoof a fake time axis since 'time' coord on full dataset is different
        # than 'time' coord on training dataset.
        simp.rename({"time": "t2"}),
        dask="parallelized",
        vectorize=True,
        # This will vectorize over the time dimension, so will submit each grid cell
        # independently
        input_core_dims=[["time"], ["time"], ["t2"]],
        # Need to denote that the final output dataset will be labeled with the
        # spoofed time coordinate
        output_core_dims=[["t2"]],
        kwargs=dict(kwargs),
    )

    # Rename to proper coordinate name.
    result = result.rename({"t2": "time"})

    # ufunc will put the core dimension to the end (time), so want to preserve original
    # order where time is commonly first.
    return result.transpose(*obs.dims)

In [12]:
group = "time.month"
method = "delta_method"



In [15]:
def grouped_adjustment(cls, method, obs, simh, simp, **kwargs):
    obs_g = list(obs.groupby("time.month"))
    simh_g = list(simh.groupby("time.month"))
    simp_g = list(simp.groupby("time.month"))
    
    result = None
    for month in range(len(list(obs_g))):
        obs_gds = obs_g[month][1]
        simh_gds = simh_g[month][1]
        simp_gds = simp_g[month][1]
        
        monthly_result = cls.apply_ufunc(method, obs_gds, simh_gds, simp_gds, kwargs)
        if result is None:
            result = monthly_result
        else:
            result = xr.concat([result, monthly_result], dim="time")
    return result

In [9]:
list(simp.groupby("time.month"))[0][1]

<xarray.Dataset>
Dimensions:  (time: 930, lat: 4, lon: 2)
Coordinates:
  * time     (time) object 2001-01-01 00:00:00 ... 2030-01-31 00:00:00
  * lat      (lat) int64 23 24 25 26
  * lon      (lon) int64 0 1
Data variables:
    tas      (time, lat, lon) float64 ...